<a href="https://colab.research.google.com/github/daniela2001v2-png/medflow/blob/main/medflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("📦 Instalando dependencias...")

!pip install -q torch torchvision torchaudio
!pip install -q transformers==4.50.0 accelerate
!pip install -q gradio pillow requests

print("✅ Dependencias instaladas!")

📦 Instalando dependencias...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 98.6 MB/s eta 0:00:00
✅ Dependencias instaladas!


In [ ]:
import torch

print("\n🔍 Verificando hardware disponible...")
print(f"GPU disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU modelo: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ No hay GPU disponible. Asegúrate de seleccionar GPU en Runtime settings.")



🔍 Verificando hardware disponible...
GPU disponible: True
GPU modelo: Tesla T4
Memoria GPU: 14.74 GB


In [ ]:
from transformers import AutoProcessor, AutoModelForImageTextToText
import time
import torch
from google.colab import userdata

print("\n🏥 Iniciando MedFlow MVP...")

MODEL_ID = "google/medgemma-4b-it"
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

print(f"⚙️ Dispositivo: {device}")
print(f"📥 Descargando Med-GEMMA 4B.")

start = time.time()

# Get the HF token from Colab secrets
HF_TOKEN = userdata.get("HF_TOKEN")

processor = AutoProcessor.from_pretrained(MODEL_ID, token=HF_TOKEN)
model = AutoModelForImageTextToText.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map=device,
    low_cpu_mem_usage=True,
    token=HF_TOKEN
)

load_time = time.time() - start
print(f"✅ Modelo cargado en {load_time:.2f} segundos!")


🏥 Iniciando MedFlow MVP...
⚙️ Dispositivo: cuda
📥 Descargando Med-GEMMA 4B.


processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

✅ Modelo cargado en 369.28 segundos!


In [ ]:
from PIL import Image

def analizar_imagen_medica(imagen, tipo_analisis, idioma, incluir_confianza=False):
    """
    Procesa imagen médica y genera reporte

    Args:
        imagen: PIL Image
        tipo_analisis: Tipo de análisis
        idioma: Español o Inglés
        incluir_confianza: Si incluir nivel de confianza

    Returns:
        tuple: (reporte, metadatos, status)
    """
    # More robust check for image validity
    if imagen is None or not isinstance(imagen, Image.Image) or imagen.size[0] == 0 or imagen.size[1] == 0:
        return "❌ Por favor carga una imagen válida", "", "Error"

    inicio = time.time()

    # Prompts según tipo de análisis
    prompts_es = {
        "Descripción General": "Describe detalladamente esta imagen médica, identificando estructuras anatómicas visibles y su apariencia.",
        "Hallazgos Patológicos": "Identifica y describe cualquier hallazgo patológico, anormal o sospechoso en esta imagen médica. Menciona ubicación, tamaño y características.",
        "Reporte Estructurado": """Genera un reporte médico estructurado siguiendo este formato:

TÉCNICA: Tipo de estudio y proyección
HALLAZGOS: Descripción detallada de hallazgos normales y anormales
IMPRESIÓN: Resumen de hallazgos principales y sugerencias""",
        "Diagnóstico Diferencial": "Basándote en los hallazgos visibles, proporciona un diagnóstico diferencial con las posibles condiciones médicas ordenadas por probabilidad."
    }

    prompts_en = {
        "Descripción General": "Describe this medical image in detail, identifying visible anatomical structures and their appearance.",
        "Hallazgos Patológicos": "Identify and describe any pathological, abnormal or suspicious findings in this medical image. Mention location, size and characteristics.",
        "Reporte Estructurado": """Generate a structured medical report following this format:

TECHNIQUE: Study type and projection
FINDINGS: Detailed description of normal and abnormal findings
IMPRESSION: Summary of main findings and suggestions""",
        "Diagnóstico Diferencial": "Based on visible findings, provide a differential diagnosis with possible medical conditions ordered by likelihood."
    }

    prompts = prompts_es if idioma == "Español" else prompts_en
    prompt = prompts.get(tipo_analisis, prompts["Descripción General"])

    if incluir_confianza:
        prompt += "\n\nAl final, indica tu nivel de confianza en el análisis (Alto/Medio/Bajo) y las limitaciones."

    # Preparar mensajes
    system_msg = "Eres un radiólogo experto." if idioma == "Español" else "You are an expert radiologist."

    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_msg}]
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {"type": "image", "image": imagen}
            ]
        }
    ]

    # Procesar
    inputs = processor.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
    ).to(device, dtype=dtype)

    input_len = inputs["input_ids"].shape[-1]

    # Generar con parámetros optimizados
    with torch.inference_mode():
        generation = model.generate(
            **inputs,
            max_new_tokens=500,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1
        )
        generation = generation[0][input_len:]

    reporte = processor.decode(generation, skip_special_tokens=True)

    tiempo = time.time() - inicio

    # Disclaimer
    disclaimer_es = """

⚠️ **DISCLAIMER MÉDICO IMPORTANTE:**
Este reporte es generado por inteligencia artificial con propósito educativo y demostrativo.
NO debe utilizarse para tomar decisiones clínicas sin supervisión médica profesional.
Este MVP es parte de un proyecto académico de la Corporación Universitaria Iberoamericana.
Siempre consulte con un radiólogo o médico especialista certificado para diagnósticos definitivos.
"""

    disclaimer_en = """

⚠️ **IMPORTANT MEDICAL DISCLAIMER:**
This report is AI-generated for educational and demonstration purposes only.
Should NOT be used for clinical decisions without professional medical supervision.
This MVP is part of an academic project from Corporación Universitaria Iberoamericana.
Always consult with a certified radiologist or specialist physician for definitive diagnoses.
"""

    disclaimer = disclaimer_es if idioma == "Español" else disclaimer_en

    # Metadatos
    metadata = f"""
📊 **Información de Procesamiento:**
- ⏱️ Tiempo: {tiempo:.2f} segundos
- 🤖 Modelo: Med-GEMMA 4B (Google Health AI)
- 💻 Dispositivo: {device.upper()}
- 🔧 Tipo de análisis: {tipo_analisis}
- 🌐 Idioma: {idioma}
- 📝 Tokens generados: {len(generation)}
- 🎯 Temperatura: 0.7
"""

    return reporte + disclaimer, metadata, f"✅ Completado en {tiempo:.2f}s"

In [ ]:
import gradio as gr

# Crear interfaz
with gr.Blocks(
    title="MedFlow MVP",
    theme=gr.themes.Soft(),
    css="""
    .gradio-container {max-width: 1200px !important}
    """
) as demo:

    gr.Markdown("""
    # 🏥 MedFlow - Producto Mínimo Viable
    ### Sistema de Interpretación Automatizada de Imágenes Médicas

    **Proyecto de Práctica Profesional - Ingeniería en Ciencia de Datos**
    **Desarrollado por:** Yeinmy Daniela Morales Barrera
    **Institución:** Corporación Universitaria Iberoamericana
    **Modelo:** Med-GEMMA 4B (Google Health AI)

    ---
    """)

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📤 Entrada")

            imagen_input = gr.Image(
                type="pil",
                label="Imagen Médica",
                height=350
            )

            gr.Markdown("**Configuración del Análisis:**")

            tipo_analisis = gr.Dropdown(
                choices=[
                    "Descripción General",
                    "Hallazgos Patológicos",
                    "Reporte Estructurado",
                    "Diagnóstico Diferencial"
                ],
                value="Reporte Estructurado",
                label="Tipo de Análisis"
            )

            idioma = gr.Radio(
                choices=["Español", "Inglés"],
                value="Español",
                label="Idioma"
            )

            incluir_confianza = gr.Checkbox(
                label="Incluir nivel de confianza",
                value=True
            )

            procesar_btn = gr.Button(
                "🔍 Analizar Imagen",
                variant="primary",
                size="lg"
            )

        with gr.Column(scale=1):
            gr.Markdown("### 📋 Resultado")

            reporte_output = gr.Textbox(
                label="Reporte Médico Generado",
                lines=16,
                placeholder="El reporte aparecerá aquí...",
                show_copy_button=True
            )

            status_output = gr.Textbox(
                label="Estado",
                lines=1,
                interactive=False
            )

    with gr.Accordion("📊 Metadatos de Procesamiento", open=False):
        metadata_output = gr.Markdown()

    gr.Markdown("""
    ---
    ### 📖 Guía de Uso:

    1. **Carga una imagen médica** utilizando el cuadro de carga (formatos: JPG, PNG, DICOM)
    2. **Selecciona el tipo de análisis:**
       - **Descripción General:** Identificación de estructuras anatómicas
       - **Hallazgos Patológicos:** Detección de anomalías
       - **Reporte Estructurado:** Formato estándar radiológico
       - **Diagnóstico Diferencial:** Posibles diagnósticos
    3. **Elige el idioma** del reporte (Español/Inglés)
    4. **Opcionalmente**, marca para incluir nivel de confianza
    5. Haz clic en **"Analizar Imagen"** y espera el resultado (~10-30 segundos)

    ### 🎯 Tipos de Imágenes Soportadas:
    - ✅ Rayos X (tórax, abdomen, extremidades)
    - ✅ Tomografías computarizadas (TC/CT)
    - ✅ Imágenes dermatológicas
    - ✅ Histopatología
    - ✅ Imágenes oftalmológicas (fundus)

    ### ⚙️ Información Técnica del MVP:
    - **Modelo Base:** Med-GEMMA 4B Multimodal Instruction-Tuned
    - **Desarrollador del Modelo:** Google Health AI
    - **Arquitectura:** Gemma 3 + SigLIP Image Encoder
    - **Entrenamiento:** Datos médicos anonimizados (MIMIC-CXR, etc.)
    - **Precisión Reportada:** 88.9 macro F1 en MIMIC-CXR
    - **Plataforma:** Google Colab con GPU T4
    - **Framework:** Transformers + PyTorch

    ### 🔬 Alcance del MVP:
    Este prototipo demuestra las capacidades básicas de interpretación de imágenes médicas.
    La versión completa de MedFlow incluirá:
    - Fine-tuning con datos colombianos
    - Integración con sistemas PACS hospitalarios
    - Cumplimiento normativo (Ley 1581, HIPAA-like)
    """)

    # Ejemplos predefinidos
    gr.Examples(
        examples=[
            [
                "https://upload.wikimedia.org/wikipedia/commons/c/c8/Chest_Xray_PA_3-8-2010.png",
                "Reporte Estructurado",
                "Español",
                True
            ],
        ],
        inputs=[imagen_input, tipo_analisis, idioma, incluir_confianza],
        label="📚 Ejemplos de Uso"
    )

    # Conectar función
    procesar_btn.click(
        fn=analizar_imagen_medica,
        inputs=[imagen_input, tipo_analisis, idioma, incluir_confianza],
        outputs=[reporte_output, metadata_output, status_output]
    )

In [10]:
print("\n" + "="*60)
print("🚀 LANZANDO MEDFLOW MVP")
print("="*60)
print("\n📱 La interfaz se abrirá en una nueva pestaña")
print("🔗 Obtendrás un link público temporal para compartir")
print("⏱️ El link expira después de 72 horas\n")

demo.launch(
    share=True,  # Genera link público temporal
    debug=True,
    show_error=True
)


🚀 LANZANDO MEDFLOW MVP

📱 La interfaz se abrirá en una nueva pestaña
🔗 Obtendrás un link público temporal para compartir
⏱️ El link expira después de 72 horas

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://56a4721a3d26577e84.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


KeyboardInterrupt: 

In [1]:
"""
MedFlow MVP - Google Colab (VERSIÓN CORREGIDA)
Fix para error CUDA device-side assert

INSTRUCCIONES:
1. Runtime > Change runtime type > T4 GPU
2. Ejecuta CELDA 1, luego CELDA 2, etc. en orden
3. La última celda lanzará la interfaz Gradio
"""

# ============================================================================
# CELDA 1: Instalación
# ============================================================================
print("📦 Instalando dependencias...")
!pip install -q transformers==4.50.0 accelerate gradio pillow requests torch
print("✅ Listo!")

# ============================================================================
# CELDA 2: Imports y Verificación
# ============================================================================
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image
import gradio as gr
import time
import traceback

print("\n🔍 Verificando GPU...")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    # Limpiar cache CUDA
    torch.cuda.empty_cache()
else:
    print("⚠️ No hay GPU. Ve a Runtime > Change runtime type > T4 GPU")

# ============================================================================
# CELDA 3: Cargar Modelo (CON FIX)
# ============================================================================
MODEL_ID = "google/medgemma-4b-it"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"\n🏥 Iniciando MedFlow...")
print(f"⚙️ Dispositivo: {device}")
print(f"📥 Descargando modelo (5-10 min primera vez)...\n")

try:
    processor = AutoProcessor.from_pretrained(MODEL_ID)

    model = AutoModelForImageTextToText.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.bfloat16,  # Cambiado a bfloat16 para mejor compatibilidad
        device_map="auto",
        low_cpu_mem_usage=True
    )

    print("✅ Modelo cargado exitosamente!\n")

except Exception as e:
    print(f"❌ Error cargando modelo: {e}")
    traceback.print_exc()

# ============================================================================
# CELDA 4: Función de Análisis (CORREGIDA)
# ============================================================================
def analizar_imagen(imagen, tipo_analisis="Reporte Estructurado", idioma="Español"):
    """
    Analiza imagen médica con Med-GEMMA
    FIX: Maneja correctamente los tensores para evitar CUDA assert error
    """

    if imagen is None:
        return "❌ Por favor carga una imagen primero", "", "Error: Sin imagen"

    try:
        inicio = time.time()

        # Prompts en español
        prompts = {
            "Descripción General": "Describe esta imagen médica identificando las estructuras anatómicas visibles.",
            "Hallazgos Patológicos": "Identifica cualquier hallazgo patológico o anormal en esta imagen médica.",
            "Reporte Estructurado": "Genera un reporte médico estructurado con: TÉCNICA, HALLAZGOS e IMPRESIÓN.",
            "Diagnóstico Diferencial": "Proporciona un diagnóstico diferencial basado en los hallazgos visibles."
        }

        prompt = prompts.get(tipo_analisis, prompts["Reporte Estructurado"])

        # Preparar mensajes
        messages = [
            {
                "role": "system",
                "content": [{"type": "text", "text": "Eres un radiólogo experto especializado en interpretación de imágenes médicas."}]
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image", "image": imagen}
                ]
            }
        ]

        # Aplicar template
        text_inputs = processor.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=False  # Importante: primero sin tokenizar
        )

        # Tokenizar por separado
        inputs = processor(
            text=text_inputs,
            images=imagen,
            return_tensors="pt",
            padding=True
        )

        # Mover a GPU de forma segura (FIX PRINCIPAL)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        input_len = inputs["input_ids"].shape[-1]

        # Generar con parámetros seguros
        print(f"🤖 Generando reporte...")

        with torch.no_grad():  # Más seguro que inference_mode
            outputs = model.generate(
                **inputs,
                max_new_tokens=350,
                do_sample=False,  # Determinístico para evitar errores
                num_beams=1,
                pad_token_id=processor.tokenizer.pad_token_id,
                eos_token_id=processor.tokenizer.eos_token_id
            )

        # Decodificar solo la parte nueva
        generated_tokens = outputs[0][input_len:]
        reporte = processor.decode(generated_tokens, skip_special_tokens=True)

        tiempo = time.time() - inicio

        # Agregar disclaimer
        disclaimer = """

⚠️ DISCLAIMER MÉDICO:
Este reporte es generado por IA con propósito educativo y demostrativo únicamente.
NO debe utilizarse para decisiones clínicas sin validación por profesionales médicos.
Proyecto académico - Corporación Universitaria Iberoamericana.
"""

        reporte_final = reporte + disclaimer

        # Metadatos
        metadata = f"""
📊 **Información de Procesamiento:**
- ⏱️ Tiempo: {tiempo:.2f} segundos
- 🤖 Modelo: Med-GEMMA 4B (Google Health AI)
- 💻 GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}
- 📝 Tokens generados: {len(generated_tokens)}
- 🔧 Tipo análisis: {tipo_analisis}
"""

        status = f"✅ Completado exitosamente en {tiempo:.2f}s"

        return reporte_final, metadata, status

    except Exception as e:
        error_msg = f"""
❌ ERROR durante el análisis:

{str(e)}

**Posibles soluciones:**
1. Verifica que tengas GPU habilitada (Runtime > Change runtime type)
2. Reinicia el runtime (Runtime > Restart runtime)
3. Intenta con una imagen más pequeña
4. Si persiste, puede ser límite de memoria - prueba cerrar otras pestañas
"""
        print(f"\n❌ Error completo:\n{traceback.format_exc()}")
        return error_msg, "Error en procesamiento", "❌ Error"

# ============================================================================
# CELDA 5: Interfaz Gradio
# ============================================================================
css = """
.gradio-container {
    max-width: 1400px !important;
    margin: auto;
}
h1 {
    text-align: center;
    color: #2563eb;
}
"""

with gr.Blocks(title="MedFlow MVP", theme=gr.themes.Soft(), css=css) as demo:

    gr.Markdown("""
    # 🏥 MedFlow - Producto Mínimo Viable
    ## Sistema de Interpretación Automatizada de Imágenes Médicas

    **Proyecto de Práctica Profesional - Ingeniería en Ciencia de Datos**
    **Desarrollado por:** Yeinmy Daniela Morales Barrera
    **Institución:** Corporación Universitaria Iberoamericana
    **Modelo:** Med-GEMMA 4B (Google Health AI)

    ---
    """)

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📤 Entrada de Datos")

            imagen_input = gr.Image(
                type="pil",
                label="📷 Imagen Médica (Rayos X, TC, etc.)",
                height=350
            )

            gr.Markdown("**Configuración:**")

            tipo_analisis = gr.Dropdown(
                choices=[
                    "Descripción General",
                    "Hallazgos Patológicos",
                    "Reporte Estructurado",
                    "Diagnóstico Diferencial"
                ],
                value="Reporte Estructurado",
                label="🔍 Tipo de Análisis",
                info="Selecciona el enfoque del análisis"
            )

            idioma = gr.Radio(
                choices=["Español", "Inglés"],
                value="Español",
                label="🌐 Idioma del Reporte"
            )

            with gr.Row():
                procesar_btn = gr.Button(
                    "🚀 Analizar Imagen",
                    variant="primary",
                    size="lg",
                    scale=2
                )
                limpiar_btn = gr.ClearButton(
                    components=[imagen_input],
                    value="🗑️ Limpiar",
                    size="lg",
                    scale=1
                )

        with gr.Column(scale=1):
            gr.Markdown("### 📋 Resultado del Análisis")

            reporte_output = gr.Textbox(
                label="Reporte Médico Generado",
                lines=18,
                placeholder="El reporte aparecerá aquí después de procesar la imagen...",
                show_copy_button=True
            )

            status_output = gr.Textbox(
                label="Estado del Proceso",
                lines=1,
                interactive=False,
                show_copy_button=False
            )

    with gr.Accordion("📊 Metadatos de Procesamiento", open=False):
        metadata_output = gr.Markdown()

    with gr.Accordion("📖 Guía de Uso Completa", open=False):
        gr.Markdown("""
        ### 🎯 Instrucciones Detalladas:

        1. **Preparar Imagen:**
           - Formatos soportados: JPG, PNG, DICOM
           - Tamaño recomendado: < 5MB
           - Resolución óptima: 512x512 a 2048x2048 píxeles

        2. **Cargar Imagen:**
           - Haz clic en el área de carga
           - Selecciona archivo o arrastra la imagen
           - Espera a que se visualice

        3. **Configurar Análisis:**
           - **Descripción General:** Identificación de estructuras anatómicas
           - **Hallazgos Patológicos:** Detección específica de anomalías
           - **Reporte Estructurado:** Formato radiológico estándar (TÉCNICA/HALLAZGOS/IMPRESIÓN)
           - **Diagnóstico Diferencial:** Lista de posibles diagnósticos

        4. **Ejecutar:**
           - Presiona "Analizar Imagen"
           - Espera 10-30 segundos según tamaño de imagen
           - Revisa el reporte generado

        ### 📚 Tipos de Imágenes Soportadas:

        | Modalidad | Ejemplos |
        |-----------|----------|
        | **Radiología** | Rayos X tórax, abdomen, extremidades |
        | **Tomografía** | TC cerebral, torácica, abdominal |
        | **Dermatología** | Lesiones cutáneas, erupciones |
        | **Patología** | Histopatología, biopsias |
        | **Oftalmología** | Fondo de ojo, retinografías |

        ### ⚙️ Especificaciones Técnicas:

        - **Modelo Base:** Med-GEMMA 4B Multimodal
        - **Arquitectura:** Gemma 3 + SigLIP Image Encoder
        - **Entrenamiento:** Datos médicos (MIMIC-CXR, PathMCQA, etc.)
        - **Precisión Reportada:** 88.9 macro F1 en clasificación MIMIC-CXR
        - **Hardware:** Google Colab T4 GPU (16GB VRAM)
        - **Framework:** PyTorch + Hugging Face Transformers

        ### 🔬 Limitaciones del MVP:

        - No reemplaza el juicio clínico profesional
        - Optimizado para imágenes en inglés (dataset base)
        - Sin fine-tuning para datos colombianos específicos
        - Sin integración con sistemas hospitalarios PACS
        - Sin validación clínica por radiólogos certificados

        ### 📈 Próximas Fases del Proyecto:

        1. **Fase 2:** Fine-tuning con datos colombianos
        2. **Fase 3:** Validación con radiólogos locales
        3. **Fase 4:** Integración PACS/RIS
        4. **Fase 5:** Cumplimiento normativo (Ley 1581 de 2012)
        """)

    # Ejemplos
    gr.Examples(
        examples=[
            ["https://upload.wikimedia.org/wikipedia/commons/c/c8/Chest_Xray_PA_3-8-2010.png", "Reporte Estructurado", "Español"],
        ],
        inputs=[imagen_input, tipo_analisis, idioma],
        label="📚 Ejemplo de Uso: Rayos X de Tórax"
    )

    # Footer
    gr.Markdown("""
    ---
    ### 📞 Información del Proyecto

    **Contacto:** ymoral35@estudiante.ibero.edu.co
    **Repositorio:** [Documentación completa disponible próximamente]
    **Versión:** MVP 1.0 (Octubre 2025)

    *Desarrollado como parte del programa de Práctica Profesional (PICD) en Ingeniería en Ciencia de Datos*
    """)

    # Conectar funciones
    procesar_btn.click(
        fn=analizar_imagen,
        inputs=[imagen_input, tipo_analisis, idioma],
        outputs=[reporte_output, metadata_output, status_output]
    )

# ============================================================================
# CELDA 6: Lanzar Aplicación
# ============================================================================
print("\n" + "="*70)
print("🚀 LANZANDO MEDFLOW MVP")
print("="*70)
print("\n✅ Todo configurado correctamente")
print("📱 La interfaz se abrirá automáticamente")
print("🔗 Compartirás un link público temporal (válido 72h)")
print("\n⚠️ IMPORTANTE: No cierres esta pestaña mientras uses la app\n")

demo.launch(
    share=True,  # Link público
    debug=True,
    show_error=True,
    server_name="0.0.0.0",
    server_port=7860
)

📦 Instalando dependencias...
✅ Listo!

🔍 Verificando GPU...
CUDA disponible: True
GPU: Tesla T4
Memoria: 14.7 GB

🏥 Iniciando MedFlow...
⚙️ Dispositivo: cuda
📥 Descargando modelo (5-10 min primera vez)...



Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Modelo cargado exitosamente!


🚀 LANZANDO MEDFLOW MVP

✅ Todo configurado correctamente
📱 La interfaz se abrirá automáticamente
🔗 Compartirás un link público temporal (válido 72h)

⚠️ IMPORTANTE: No cierres esta pestaña mientras uses la app

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d9fa7a17e312fd87bc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🤖 Generando reporte...
🤖 Generando reporte...
Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://d9fa7a17e312fd87bc.gradio.live
